# Wikidata にある日本の市区町村データの整合性チェック

Wikidata にある日本の市区町村データと、総務省サイトにあるそれとを比較し、一致することを確認する。

In [28]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON, XML

# 表のすべての行を表示する
pd.set_option('display.max_rows', None)

## 総務省サイトからすべての市区町村の一覧を取得

In [51]:
# 総務省のサイト https://www.soumu.go.jp/denshijiti/code.html から
# 最新の「都道府県コード及び市区町村コード」を読み込む。
excel_file_url = "https://www.soumu.go.jp/main_content/000894847.xls"

# １枚目のシート。市町村と東京特別区。
df_municipalities = pd.read_excel(excel_file_url, sheet_name=0)
# カラム名を変更
df_municipalities.rename(columns={'都道府県名\n（漢字）': '都道府県名', '市区町村名\n（漢字）': '市区町村名'}, inplace=True)
# 市区町村名のカラムが　NaN の行は都道府県なので削除する
df_municipalities = df_municipalities.dropna(subset=['市区町村名'])
df_municipalities # これは市町村と東京都の区を含む

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）
1,11002,北海道,札幌市,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼ
2,12025,北海道,函館市,ﾎｯｶｲﾄﾞｳ,ﾊｺﾀﾞﾃｼ
3,12033,北海道,小樽市,ﾎｯｶｲﾄﾞｳ,ｵﾀﾙｼ
4,12041,北海道,旭川市,ﾎｯｶｲﾄﾞｳ,ｱｻﾋｶﾜｼ
5,12050,北海道,室蘭市,ﾎｯｶｲﾄﾞｳ,ﾑﾛﾗﾝｼ
6,12068,北海道,釧路市,ﾎｯｶｲﾄﾞｳ,ｸｼﾛｼ
7,12076,北海道,帯広市,ﾎｯｶｲﾄﾞｳ,ｵﾋﾞﾋﾛｼ
8,12084,北海道,北見市,ﾎｯｶｲﾄﾞｳ,ｷﾀﾐｼ
9,12092,北海道,夕張市,ﾎｯｶｲﾄﾞｳ,ﾕｳﾊﾞﾘｼ
10,12106,北海道,岩見沢市,ﾎｯｶｲﾄﾞｳ,ｲﾜﾐｻﾞﾜｼ


In [83]:
# ２枚目のシート。普通の区。
df_wards = pd.read_excel(excel_file_url, sheet_name=1)
# カラム名を変更
df_wards.rename(columns={'都道府県名\n（漢字）': '都道府県名', '市区町村名\n（漢字）': '市区町村名'}, inplace=True)
# 市区町村名のカラムが「市」で終わる行は市なので削除する
df_wards = df_wards.query('not 市区町村名.str.endswith("市")')
df_wards

,団体コード,都道府県名,市区町村名,都道府県名\n（ｶﾅ）,市区町村名\n（ｶﾅ）,Unnamed: 5
1,11011,北海道,札幌市中央区,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾁｭｳｵｳｸ,NaN
2,11029,北海道,札幌市北区,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼｷﾀｸ,NaN
3,11037,北海道,札幌市東区,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾋｶﾞｼｸ,NaN
4,11045,北海道,札幌市白石区,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼｼﾛｲｼｸ,NaN
5,11053,北海道,札幌市豊平区,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾄﾖﾋﾗｸ,NaN
6,11061,北海道,札幌市南区,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾐﾅﾐｸ,NaN
7,11070,北海道,札幌市西区,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾆｼｸ,NaN
8,11088,北海道,札幌市厚別区,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼｱﾂﾍﾞﾂｸ,NaN
9,11096,北海道,札幌市手稲区,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼﾃｲﾈｸ,NaN
10,11100,北海道,札幌市清田区,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼｷﾖﾀｸ,NaN


Wikidata から SPARQL 言語による検索結果を RDF(JSON) で得る関数を定義する。

In [52]:
def QuerySparql(query):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.queryAndConvert()
    return results['results']['bindings']

## 市のチェック

### Wikidata から市のリストを JSON で取得

In [53]:

cities_query = """
SELECT DISTINCT ?city ?cityLabel ?code ?pref ?prefLabel WHERE {
  ?city wdt:P31/wdt:P279* wd:Q494721 . # ?city は「日本の市」またはその部分集合（の部分集合……）の要素である。
  MINUS { ?city wdt:P576 ?abolished . } # ただし「廃止日」がないこと。
  OPTIONAL { ?city wdt:P429 ?code . } # 全国地方公共団体コード
  OPTIONAL {
    # P131 は「位置する行政区画」を表す。
    # 以下のような例で正しく都道府県を得るために、wdt:P131 のあとに '+' が必要。
    # 札幌市 P131 石狩振興局 P131 北海道
    ?city wdt:P131+ ?pref. # ?pref に位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "ja" . }
}
ORDER BY ASC(?code)
"""
json_cities_wikidata = QuerySparql(cities_query)
json_cities_wikidata

[{'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q37951'},
  'cityLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '札幌市'},
  'code': {'type': 'literal', 'value': '011002'},
  'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1037393'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '北海道'}},
 {'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q26418'},
  'cityLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '函館市'},
  'code': {'type': 'literal', 'value': '012025'},
  'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1037393'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '北海道'}},
 {'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q242108'},
  'cityLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '小樽市'},
  'code': {'type': 'literal', 'value': '012033'},
  'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1037393'},
  'prefLabel': {'xml:lang': '

### pandas DataFrame へ変換

In [54]:
cities_wikidata = []
for c in json_cities_wikidata:
    city = {
        '団体コード': int(c['code']['value']),
        '都道府県名': (c['prefLabel']['value'] if ('prefLabel' in c) else None),
        '市区町村名': c['cityLabel']['value']
    }
    cities_wikidata.append(city)
df_cities_wikidata = pd.DataFrame(cities_wikidata)
df_cities_wikidata

,団体コード,都道府県名,市区町村名
0,11002,北海道,札幌市
1,12025,北海道,函館市
2,12033,北海道,小樽市
3,12041,北海道,旭川市
4,12050,北海道,室蘭市
5,12068,北海道,釧路市
6,12076,北海道,帯広市
7,12084,北海道,北見市
8,12092,北海道,夕張市
9,12106,北海道,岩見沢市


この表で都道府県名が Noneになっている市がある場合、市の「位置する行政区画（P131）」またはその1回以上の繰り返しに都道府県が存在しないので、Wikidata 上で修正すること。

### 総務省の市町区村の一覧から市だけを得る

In [55]:
df_cities = df_municipalities.query('市区町村名.str.endswith("市")')
df_cities

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）
1,11002,北海道,札幌市,ﾎｯｶｲﾄﾞｳ,ｻｯﾎﾟﾛｼ
2,12025,北海道,函館市,ﾎｯｶｲﾄﾞｳ,ﾊｺﾀﾞﾃｼ
3,12033,北海道,小樽市,ﾎｯｶｲﾄﾞｳ,ｵﾀﾙｼ
4,12041,北海道,旭川市,ﾎｯｶｲﾄﾞｳ,ｱｻﾋｶﾜｼ
5,12050,北海道,室蘭市,ﾎｯｶｲﾄﾞｳ,ﾑﾛﾗﾝｼ
6,12068,北海道,釧路市,ﾎｯｶｲﾄﾞｳ,ｸｼﾛｼ
7,12076,北海道,帯広市,ﾎｯｶｲﾄﾞｳ,ｵﾋﾞﾋﾛｼ
8,12084,北海道,北見市,ﾎｯｶｲﾄﾞｳ,ｷﾀﾐｼ
9,12092,北海道,夕張市,ﾎｯｶｲﾄﾞｳ,ﾕｳﾊﾞﾘｼ
10,12106,北海道,岩見沢市,ﾎｯｶｲﾄﾞｳ,ｲﾜﾐｻﾞﾜｼ


### Wikidata にあって総務省のデータにはない市を表示

何もないのが正しい。

In [56]:
cities_only_in_wikidata = df_cities_wikidata[~df_cities_wikidata['団体コード'].isin(df_cities['団体コード'])]
cities_only_in_wikidata

,団体コード,都道府県名,市区町村名


### 総務省のデータにあって Wikidata にはない市を表示

何もないのが正しい。

In [80]:
cities_not_in_wikidata = df_cities[~df_cities['団体コード'].isin(df_cities_wikidata['団体コード'])]
cities_not_in_wikidata

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）


### 市の数のチェック

In [64]:
# 都道府県ごとの市の数
num_cities_query = """
SELECT ?pref ?prefLabel ?prefCode (COUNT(DISTINCT ?city) AS ?count) WHERE {
  ?city wdt:P31/wdt:P279* wd:Q494721 . # ?city は「日本の市」またはその部分集合（の部分集合……）の要素である。
  MINUS { ?city wdt:P576 ?abolished. } # ただし「廃止日」がないこと。
  OPTIONAL {
    # P131 は「位置する行政区画」を表す。
    # 以下のような例で正しく都道府県を得るために、wdt:P131 のあとに '+' が必要。
    # 札幌市 P131 石狩振興局 P131 北海道
    ?city wdt:P131+ ?pref. # ?pref に位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
  }
  OPTIONAL { ?pref wdt:P429 ?prefCode. } # 都道府県の全国地方公共団体コード
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "ja" .
  }
}
GROUP BY ?pref ?prefLabel ?prefCode
ORDER BY ASC(?prefCode)
"""
json_num_cities_wikidata = QuerySparql(num_cities_query)
json_num_cities_wikidata

[{'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1037393'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '北海道'},
  'prefCode': {'type': 'literal', 'value': '010006'},
  'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
   'type': 'literal',
   'value': '35'}},
 {'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q71699'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '青森県'},
  'prefCode': {'type': 'literal', 'value': '020001'},
  'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
   'type': 'literal',
   'value': '10'}},
 {'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q48326'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '岩手県'},
  'prefCode': {'type': 'literal', 'value': '030007'},
  'count': {'datatype': 'http://www.w3.org/2001/XMLSchema#integer',
   'type': 'literal',
   'value': '14'}},
 {'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entit

In [65]:
# paandas DataFrame へ変換
num_cities_wikidata = []
for c in json_num_cities_wikidata:
    city = {
        '都道府県名': c['prefLabel']['value'],
        '市の数': int(c['count']['value'])
    }
    num_cities_wikidata.append(city)
df_num_cities_wikidata = pd.DataFrame(num_cities_wikidata)
df_num_cities_wikidata

,都道府県名,市の数
0,北海道,35
1,青森県,10
2,岩手県,14
3,宮城県,14
4,秋田県,13
5,山形県,13
6,福島県,13
7,茨城県,32
8,栃木県,14
9,群馬県,12


これらの　Wikidata による市の数を、総務省のページ
https://www.j-lis.go.jp/spd/code-address/kenbetsu-inspection/cms_11914151.html
にある数と比べ、一致することを確認すること。

## 東京都の区のチェック

In [77]:
# Wikidata から特別区のリストを取得
tokyo_wards_query = """
SELECT DISTINCT ?ward ?wardLabel ?code ?city ?cityLabel ?pref ?prefLabel WHERE {
  ?ward wdt:P31 wd:Q5327704 . # ?ward は「日本の特別区」の要素である。
  MINUS { ?ward wdt:P576 ?abolished . } # ただし「廃止日」がないこと。
  OPTIONAL { ?ward wdt:P429 ?code . } # 全国地方公共団体コード
  OPTIONAL {
    # P131 は「位置する行政区画」を表す。
    ?ward wdt:P131+ ?pref. # ?ward は ?pref に位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "ja" . }
}
ORDER BY ASC(?code)
"""
json_tokyo_wards_wikidata = QuerySparql(tokyo_wards_query)
json_tokyo_wards_wikidata

[{'ward': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q214051'},
  'wardLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '千代田区'},
  'code': {'type': 'literal', 'value': '131016'},
  'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1490'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '東京都'}},
 {'ward': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q212704'},
  'wardLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '中央区'},
  'code': {'type': 'literal', 'value': '131024'},
  'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1490'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '東京都'}},
 {'ward': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q190088'},
  'wardLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '港区'},
  'code': {'type': 'literal', 'value': '131032'},
  'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1490'},
  'prefLabel': {'xml:lang': 'ja', 't

In [78]:
# pandas DataFrame へ変換
tokyo_wards_wikidata = []
for item in json_tokyo_wards_wikidata:
    ward = {
        '団体コード': (int(item['code']['value']) if ('code' in item) else -1),
        '区名': item['wardLabel']['value'],
        '都道府県名': (item['prefLabel']['value'] if ('prefLabel' in item) else None)
    }
    tokyo_wards_wikidata.append(ward)
df_tokyo_wards_wikidata = pd.DataFrame(tokyo_wards_wikidata)
df_tokyo_wards_wikidata

,団体コード,区名,都道府県名
0,131016,千代田区,東京都
1,131024,中央区,東京都
2,131032,港区,東京都
3,131041,新宿区,東京都
4,131059,文京区,東京都
5,131067,台東区,東京都
6,131075,墨田区,東京都
7,131083,江東区,東京都
8,131091,品川区,東京都
9,131105,目黒区,東京都


In [79]:
# 総務省の市町区村の一覧から東京都の区を得る
df_tokyo_wards = df_municipalities.query('市区町村名.str.endswith("区")')
df_tokyo_wards

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）
646,131016,東京都,千代田区,ﾄｳｷｮｳﾄ,ﾁﾖﾀﾞｸ
647,131024,東京都,中央区,ﾄｳｷｮｳﾄ,ﾁｭｳｵｳｸ
648,131032,東京都,港区,ﾄｳｷｮｳﾄ,ﾐﾅﾄｸ
649,131041,東京都,新宿区,ﾄｳｷｮｳﾄ,ｼﾝｼﾞｭｸｸ
650,131059,東京都,文京区,ﾄｳｷｮｳﾄ,ﾌﾞﾝｷｮｳｸ
651,131067,東京都,台東区,ﾄｳｷｮｳﾄ,ﾀｲﾄｳｸ
652,131075,東京都,墨田区,ﾄｳｷｮｳﾄ,ｽﾐﾀﾞｸ
653,131083,東京都,江東区,ﾄｳｷｮｳﾄ,ｺｳﾄｳｸ
654,131091,東京都,品川区,ﾄｳｷｮｳﾄ,ｼﾅｶﾞﾜｸ
655,131105,東京都,目黒区,ﾄｳｷｮｳﾄ,ﾒｸﾞﾛｸ


In [81]:
# Wikidata にあって総務省のデータにはないものを表示
# 何もないのが正しい。
tokyo_wards_only_in_wikidata = df_tokyo_wards_wikidata[~df_tokyo_wards_wikidata['団体コード'].isin(df_tokyo_wards['団体コード'])]
tokyo_wards_only_in_wikidata

,団体コード,区名,都道府県名


In [82]:
# 総務省のデータにあって Wikidata にはない市を表示
# 何もないのが正しい。
tokyo_wards_not_in_wikidata = df_tokyo_wards[~df_tokyo_wards['団体コード'].isin(df_tokyo_wards_wikidata['団体コード'])]
tokyo_wards_not_in_wikidata

,団体コード,都道府県名,市区町村名,都道府県名\n（カナ）,市区町村名\n（カナ）


## 区のチェック

In [86]:
# Wikidata から区のリストを取得
wards_query = """
SELECT DISTINCT ?ward ?wardLabel ?code ?city ?cityLabel ?pref ?prefLabel WHERE {
  ?ward wdt:P31/wdt:P279* wd:Q137773 . # ?ward は「日本の区」またはその部分集合（の部分集合……）の要素である。
  MINUS { ?ward wdt:P576 ?abolished . } # ただし「廃止日」がないこと。
  OPTIONAL { ?ward wdt:P429 ?code . } # 全国地方公共団体コード
  OPTIONAL {
    # P131 は「位置する行政区画」を表す。
    ?ward wdt:P131+ ?city. # _ward は ?city に位置する。
    ?city wdt:P31 wd:Q494721. # ?city は「日本の市」である。
    ?city wdt:P131+ ?pref. # ?city は ?pref に位置する。
    ?pref wdt:P31 wd:Q50337. # ?pref は「都道府県」である。
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "ja" . }
}
ORDER BY ASC(?code)
"""
json_wards_wikidata = QuerySparql(wards_query)
json_wards_wikidata

[{'ward': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1079635'},
  'wardLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '中央区'},
  'code': {'type': 'literal', 'value': '011011'},
  'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q37951'},
  'cityLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '札幌市'},
  'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1037393'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '北海道'}},
 {'ward': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1134013'},
  'wardLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '北区'},
  'code': {'type': 'literal', 'value': '011029'},
  'city': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q37951'},
  'cityLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '札幌市'},
  'pref': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1037393'},
  'prefLabel': {'xml:lang': 'ja', 'type': 'literal', 'value': '北海道'}},
 {'ward':

In [87]:
# pandas DataFrame へ変換
wards_wikidata = []
for w in json_wards_wikidata:
    ward = {
        '団体コード': (int(w['code']['value']) if ('code' in w) else -1),
        '区名': w['wardLabel']['value'],
        '市名': w['cityLabel']['value'],
        '都道府県名': (w['prefLabel']['value'] if ('prefLabel' in w) else None)
    }
    wards_wikidata.append(ward)
df_wards_wikidata = pd.DataFrame(wards_wikidata)
df_wards_wikidata

,団体コード,区名,市名,都道府県名
0,11011,中央区,札幌市,北海道
1,11029,北区,札幌市,北海道
2,11037,東区,札幌市,北海道
3,11045,白石区,札幌市,北海道
4,11053,豊平区,札幌市,北海道
5,11061,南区 (札幌市),札幌市,北海道
6,11070,西区 (札幌市),札幌市,北海道
7,11088,厚別区,札幌市,北海道
8,11096,手稲区,札幌市,北海道
9,11100,清田区,札幌市,北海道


この表で「団体コード」や「都道府県名」が -1 や Noneになっている行がある場合、必要に応じて Wikidata 上で元データを修正すること。

In [88]:
# 団体コードが -1 のものを除外。新設される予定で、まだコードが振られていない区などがそれに当たる。
df_wards_wikidata = df_wards_wikidata[df_wards_wikidata['団体コード'] != -1]
df_wards_wikidata

,団体コード,区名,市名,都道府県名
0,11011,中央区,札幌市,北海道
1,11029,北区,札幌市,北海道
2,11037,東区,札幌市,北海道
3,11045,白石区,札幌市,北海道
4,11053,豊平区,札幌市,北海道
5,11061,南区 (札幌市),札幌市,北海道
6,11070,西区 (札幌市),札幌市,北海道
7,11088,厚別区,札幌市,北海道
8,11096,手稲区,札幌市,北海道
9,11100,清田区,札幌市,北海道


In [89]:
# Wikidata にあって総務省のデータにはないものを表示
# 何もないのが正しい。
# 2023/10/01 現在、浜松市の2024年に予定されている統合前の区がここに表示される。
wards_only_in_wikidata = df_wards_wikidata[~df_wards_wikidata['団体コード'].isin(df_wards['団体コード'])]
wards_only_in_wikidata

,団体コード,区名,市名,都道府県名
70,221317,中区,浜松市,静岡県
71,221325,東区,浜松市,静岡県
72,221333,西区,浜松市,静岡県
73,221341,南区,浜松市,静岡県
74,221350,北区,浜松市,静岡県
75,221368,浜北区,浜松市,静岡県
76,221376,天竜区,浜松市,静岡県


In [90]:
# 総務省のデータにあって Wikidata にはない市を表示
# 何もないのが正しい。
# 2023/10/01 現在、浜松市の2024年に予定されている統合で団体コードがつけ変わる天竜区がここに表示される。
wards_not_in_wikidata = df_wards[~df_wards['団体コード'].isin(df_wards_wikidata['団体コード'])]
wards_not_in_wikidata

,団体コード,都道府県名,市区町村名,都道府県名\n（ｶﾅ）,市区町村名\n（ｶﾅ）,Unnamed: 5
82,221406,静岡県,浜松市天竜区,ｼｽﾞｵｶｹﾝ,ﾊﾏﾏﾂｼﾃﾝﾘｭｳｸ,R6.1.1施行
